[ZZAEBOK'S BLOG](https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/)  
해당 글을 보고 따라함. Windows 환경에서 구현했으며 약간의 문제점이 발생했음(해결)

네이버 영화 리뷰 corpus의 감성 이진분류(긍정/부정)가 목적  
Huggingface Transformer를 참고하여 작성하였다 한다.

In [1]:
# pytorch-transformers 라이브러리 설치
!pip install transformers

In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from IPython.display import clear_output

In [3]:
# 필요한 데이터셋 다운로드
!git clone https://github.com/e9t/nsmc.git
clear_output()

해당 블로그는 데이터를 40%만 사용. 현재 gpu는 1660s 6GB이므로 사용해보고 판단 결정

In [4]:
train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t') # 15만개
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t') # 5만
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

# 랜덤하게 데이터 추출
# train_df = train_df.sample(frac=0.4, random_state=999)
# test_df = test_df.sample(frac=0.4, random_state=999)
train_df.tail()

,id,document,label
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1
149999,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [5]:
class NsmcDataset(Dataset):
    '''Naver Sentiment Movie Corpus Dataset'''
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label    

In [6]:
# dataloader 생성
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=0) # window에서는 num_workers=0으로 해야한다.

In [7]:
device = torch.device('cuda')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [8]:
tokenizer.tokenize('이거 왜이렇게 성능이 별로냐')

['이', '##거', '왜', '##이', '##렇게', '성', '##능', '##이', '별', '##로', '##냐']

In [9]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.load_state_dict(torch.load('./results/nsmc_model.pth'))
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

실험  
Huggingface transformer에서 제공하는 Trainer를 돌려본다.

In [10]:
# 안됨
nsmc_eval_dataset = NsmcDataset(test_df)

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=nsmc_train_dataset,
    eval_dataset=nsmc_eval_dataset
)

In [11]:
# trainer.train()

In [12]:
# optimizer = Adam(model.parameters(), lr=1e-6)


# itr = 1
# p_itr = 500
# epochs = 1
# total_loss = 0
# total_len = 0
# total_correct = 0

# for epoch in range(epochs):
#     for text, label in train_loader:
#         optimizer.zero_grad()
        
#         # encoding and zero padding
#         encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text] # text to integer
#         padded_list = [e + [0] * (512-len(e)) for e in encoded_list] # padding을 뒤에 추가
        
#         sample = torch.tensor(padded_list)
#         sample, label = sample.to(device), label.to(device)
        
#         labels = torch.tensor(label)
#         outputs = model(sample, labels=labels)
#         loss, logits = outputs
        
#         pred = torch.argmax(F.softmax(logits), dim=1)
#         correct = pred.eq(labels)
#         total_correct += correct.sum().item()
#         total_len += len(labels)
#         total_loss += loss.item()
#         loss.backward()
#         optimizer.step()
        
#         if itr % p_itr == 0:
#             print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
#             total_loss = 0
#             total_len = 0
#             total_correct = 0

#         itr+=1

# torch.save(model.state_dict(), './results/nsmc_model.pth')

### 모델 평가

In [13]:
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test accuracy:  0.8514310858651519


In [14]:
total_correct / total_len

0.8514310858651519